### Confusion matrix

||예측P|예측N||
|:---:|:---:|:---:|:---:|
|실제P|TP|FN|Recall(Sensitivity)|
|실제N|FP|TN|Specificity|
||Precision|NegPredVal|Accuracy|



##### Accuracy = 예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수
* 정확도는 직관적으로 모델 예측 성능을 나타내는 평가 지표.  
하지만 이진 분류의 경우 데이터의 구성에 따라 ML 모형의 성능 왜곡할 수 있음.   
정확도 수치 하나만 가지고 성능을 평가하지 않음
* 특히 정확도는 imbalanced 레이블 값 분포하에서는 적합하지 않은 지표

##### Recall = TP / TP + FN (예측과 실제가 모두 참 / 실제 참인 값)
* 재현율은 실제 값이 P인 대상 중에 예측과 실제값이 P로 일치한 데이터의 비율
* 병이 있을 때 있다고 판단한 비율

##### Precision = TP / TP + FP (예측과 실제가 모두 참 / 참으로 예측한 값)
* 정밀도는 예측을 P로 한 대상 중에 예측과 실제 값이 P로 일치한 데이터의 비율

##### Specificity = TN / FP + TN (예측과 실제가 모두 거짓 / 실제 거짓인 값)
* 병이 없을 때 없다고 판단한 비율

##### 정밀도와 재현율의 Trade Off
- 정밀도와 재현율이 강조될 경우 Threshhold를 조정해 해당 수치 조정 가능
- 상호 보완적인 수치이므로 Trade Off 작용
- 모든 환자를 양성으로 판정 → FN = TN = , Recall = 100%
- 확실한 경우만 양성, 나머지 모두 N → FP = 0, TP = 1

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred) :
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print("Confusion Matrix")
    print(confusion)
    print(f"Accuracy : {accuracy:.4f}, Precision : {precision:.4f}, Recall : {recall:.4f}")

In [ ]:
import numpy as np
import pandas as pd
from modules import DtPre

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

titanic_df = pd.read_csv("./csv_data/titanic_train.csv")
y_titanic_df = titanic_df["Survived"]   # 레이블 데이터 셋 추출
X_titanic_df = titanic_df.drop("Survived", axis=1)  # 피쳐 데이터 셋에서 레이블셋은 삭제

X_titanic_df = DtPre.transform_features(X_titanic_df) # 만들어둔 전처리 함수 적용

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,
                                                    test_size=0.2, random_state=11)

lr_clf = LogisticRegression()

lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

#  TN    FP  실제 N
#  FN    TP  실제 P
# 예측N 예측P

#### ROC & AUC
* TPR = TP / TP + FN (= Recall)  /  TPR은 1에 가까울 수록 좋다
* FPR = FP / FP + TN (= 1-Specificity)  /  FPR은 0에 가까울 수록 좋다
* Decision Threshold를 높인다면 P로 분류되는 경우가 작아짐   
만약 DT = 1 → TPR = FPR = 0     
만약 DT = 0 → TPR = FPR = 1
* 따라서 DT에 의해 Trade Off
* ROC는 (FPR, TPR)쌍을 평면상에 그림    
AUC는 ROC 아래 면적 → 1에 가까울 수록 좋은 수치     
AUC = 1 → TPR = 1, FPR = 0

##### F1 Score
* 정밀도와 재현율을 결합한 지표, 어느 한쪽으로 치우치지 않는 수치일 때   
상대적으로 높은 값을 가짐 : 기하평균
* F1 = 2 / 1/recall + 1/precision = 2(precision*recall)/precision + recall

## 정확도의 함정
* 더미 분류기 만들어도 데이터의 성질 또는 분포에 따라 정확도 높을 수 있음

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator) :
    # fit() 메소드는 아무것도 학습하지 않음
    def fit(self, X, y=None) :
        pass
    # predict() 메소드는 단순히 Sex feature가 1 이면 0, 그렇지 않으면 1로 예측함
    def predict(self, X) :
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]) :
            if X["Sex"].iloc[i] == 1 :
                pred[i] = 0
            else :
                pred[i] = 1
                
        return pred

In [ ]:
# 위에서 생성한 DummyClassifier에 적용
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print(f"DummyClassifier의 정확도는 : {accuracy_score(y_test, mypredictions):.4f}")
# 아무런 학습도 시키지 않고 단순히 여자면 살았다고 판단하기만 해도 83%정확도

In [ ]:
from sklearn.datasets import load_digits

class MyFakeClassifier(BaseEstimator) :
    # 학습 안함
    def fit(self, X, y) :
        pass

    # 입력값으로 들어오는 X 데이터 셋의 크기만큼 0 행렬로 반환
    def predict(self, X) :
        return np.zeros((len(X), 1), dtype=bool)

# 사이킷런의 내장 데이터 셋인 load_digits()를 이용하여 MNIST 데이터 로딩
digits = load_digits()

print(digits.data)
print(f"### digits.data.shape : {digits.data.shape}")
print(digits.target)
print(f"###digits.target.shape : {digits.target.shape}")

In [ ]:
digits.target == 7

In [ ]:
# digits 번호가 7번이면 True, 이를 astype(int)로 1로 변환, 7이 아니면 False, 0
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y,
                                                    random_state=11)

In [ ]:
# 불균형한 레이블 데이터 분포도 확인
print(f"레이블 테스트 세트 크기 : {y_test.shape}")
print(f"테스트 세트 레이블의 0과 1의 분포도")
print(pd.Series(y_test).value_counts())

# FakeClassifier 적용
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print(f"모든 예측을 0으로 하여도 정확도는 : {accuracy_score(y_test, fakepred):.3f}")

In [ ]:
# 앞절의 예측 결과인 fakepred와 실제 결과인 y_test의 Confusion Matrix 출력
confusion_matrix(y_test, fakepred)

In [ ]:
# 정확도 높지만 정밀도와 재현율 0
print(f"정밀도 : {precision_score(y_test, fakepred)}")
print(f"재현율 : {recall_score(y_test, fakepred)}")

### 분류 결정 임곗값에 따른 Positive 예측 확률 변화

In [ ]:
titanic_df = pd.read_csv("./csv_data/titanic_train.csv")
y_titanic_df = titanic_df["Survived"]   # 레이블 데이터 셋 추출
X_titanic_df = titanic_df.drop("Survived", axis=1)  # 피쳐 데이터 셋에서 레이블셋은 삭제

X_titanic_df = DtPre.transform_features(X_titanic_df) # 만들어둔 전처리 함수 적용

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,
                                                    test_size=0.2, random_state=11)

lr_clf = LogisticRegression()

lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

In [ ]:
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)
print(f"pred_proba()결과 Shape : {pred_proba.shape}")
print(f"pred_proba array에서 앞 5개만 샘플로 추출 : \n {pred_proba[:5]}")

# 예측 확률 array와 예측 결과값 array를 concatenate하여
# 예측 확률과 결과값을 한눈에 확인
# pred는 [1,0,0...] 형태의 array
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1,1)], axis=1)
# 앞이 0, 뒤가 1이 될 확률
print(f"두개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n {pred_proba_result[:5]}")


In [ ]:
from sklearn.preprocessing import Binarizer

X = [[1, -1, 2],
     [2, 0, 0],
     [0, 1.1, 1.2]]

# threshold 기준값보다 같거나 작으면 0을, 크면 1을 반환  /  초과
binarizer = Binarizer(threshold=1.1)
print(binarizer.fit_transform(X))

In [ ]:
# Binarizer의 threshold 설정값, 분류 결정 임곗값
custom_threshold = 0.5

# predict_proba() 반환값의 두번째 컬럼, 즉 Positive 클래스 컬럼 하나만 추출하여
# Binarizer 적용  /  Positive 일 확률이 0.5를 초과하는가?
pred_proba_1 = pred_proba[:, 1].reshape(-1,1)

binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_1) # 확률검사
custom_predict = binarizer.transform(pred_proba_1)  # 0, 1 변환

get_clf_eval(y_test, custom_predict)
# 앞과 동일한 결과

In [ ]:
# Binarizer의 threshold 설정값을 0.4로 변경. 분류 결정 임곗값을 낮춤.
custom_threshold = 0.4

pred_proba_1 = pred_proba[:, 1].reshape(-1,1)

binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_1) # 확률검사
custom_predict = binarizer.transform(pred_proba_1)  # 0, 1 변환

get_clf_eval(y_test, custom_predict)
# 재현율 높아짐 / 정밀도 낮아짐

##### 정밀도와 재현율의 Trade Off
- 정밀도와 재현율이 강조될 경우 Threshhold를 조정해 해당 수치 조정 가능
- 상호 보완적인 수치이므로 Trade Off 작용
- 모든 환자를 양성으로 판정 → FN = TN = , Recall = 100%
- 확실한 경우만 양성, 나머지 모두 N → FP = 0, TP = 1

#### ROC & AUC
* TPR = TP / TP + FN (= Recall)  /  TPR은 1에 가까울 수록 좋다
* FPR = FP / FP + TN (= 1-Specificity)  /  FPR은 0에 가까울 수록 좋다
* Decision Threshold를 높인다면 P로 분류되는 경우가 작아짐   
만약 DT = 1 → TPR = FPR = 0     
만약 DT = 0 → TPR = FPR = 1
* 따라서 DT에 의해 Trade Off
* ROC는 (FPR, TPR)쌍을 평면상에 그림    
AUC는 ROC 아래 면적 → 1에 가까울 수록 좋은 수치     
AUC = 1 → TPR = 1, FPR = 0

In [ ]:
# 테스트를 수행할 모든 임곗값을 리스트 객체로 저장
thresholds = [0.4,0.45, 0.5, 0.55, 0.6]

def get_eval_by_threshold(y_test, pred_proba_c1, thresholds) :
    # thresholds list 객체 내의 값을 차례로 iteration 하면서 Evaluation 수행
    for custom_threshold in thresholds :
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print(f"임곗값 : {custom_threshold}")
        get_clf_eval(y_test, custom_predict)

get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)
# 임곗값 상승할 수록 재현율 하락, 정밀도 상승 trade off

In [ ]:
from sklearn.metrics import precision_recall_curve

# 레이블 값이 1일때의 예측 확률을 추출
pred_proba_class1 = lr_clf.predict_proba(X_test)[:,1]

# 실제값 데이터 셋과 레이블 값이 1일 때의 예측 확률을
# precision_recall_curve 인자로 입력
precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_class1)
print(f"반환된 분류 결정 임곗값 배열의 Shape : {thresholds.shape}")
print(f"반환된 precision 배열의 Shape : {precisions.shape}")
print(f"반환된 recalls 배열의 Shape : {recalls.shape}")

print(f"thresholds 5 samples : {thresholds[:5]}")
print(f"precisions 5 samples : {precisions[:5]}")
print(f"recalls 5 samples : {recalls[:5]}")

# 반환된 임계값 배열 행이 147건이므로 샘플 10건만 추출, 임곗값을 15 steps로 추출
thr_index = np.arange(0, thresholds.shape[0], 15)
print(f"샘플 추출을 위한 임곗값 배열의 index 10개 : {thr_index}")
print(f"샘플용 10개의 임곗값 : {np.round(thresholds[thr_index], 2)}")

# 15 steps 단위로 추출된 임곗값에 따른 정밀도와 재현율 값
print(f"샘플 임곗값별 정밀도 : {np.round(precisions[thr_index],3)}")
print(f"샘플 임곗값별 재현율 : {np.round(recalls[thr_index], 3)}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def precision_recall_curve_plot(y_test, pred_proba_c1) :
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_c1)

    # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 plot 수행.
    # 정밀도는 점선으로 표시
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle="--", label="precision")
    plt.plot(thresholds, recalls[0:threshold_boundary], label="recall")

    # threshold값 X 축의 Scale을 0.1 단위로 변경
    start, end = plt.xlim()
    # thresholds의 값을 0.10394, 0.2~ 이런식으로 배열하고 소숫점 2자리 반올림
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))

    # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel("Threshold value"); plt.ylabel("Precision and Recall value")
    plt.legend(); plt.grid()
    plt.show()

precision_recall_curve_plot(y_test, lr_clf.predict_log_proba(X_test)[:,1])

## 사이킷런 ROC 곡선 및 AUC 스코어

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, pred)
print(f"F1 scores : {f1:.4f}")

In [ ]:
def get_clf_eval(y_test, pred) :
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    # F1 스코어 추가
    f1 = f1_score(y_test, pred)
    print("Confusion Matrix")
    print(confusion)
    # f1 score print 추가
    print(f"Accuracy : {accuracy:.4f}, Precision : {precision:.4f}, Recall : {recall:.4f}, F1 :{f1:.4f}")

In [ ]:
thresholds = [0.4,0.45, 0.5, 0.55, 0.6]
pred_proba = lr_clf.predict_proba(X_test)
get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)
# 임곗값의 변화와 재현율, 정밀도에 따라 F1 스코어 변동

In [ ]:
from sklearn.metrics import roc_curve

# 레이블 값이 1일때의 예측 확률을 추출
pred_proba_class1 = lr_clf.predict_proba(X_test)[:,1]

fprs, tprs, thresholds = roc_curve(y_test, pred_proba_class1)
# 반환된 임곗값 배열에서 샘플로 데이터를 추출하되, 임곗값을 5 steps로 추출
# thresholds[0]는 max(예측확률)+1로 임의 설정.
# 이를 제외하기 위해 np.arange 는 1부터 시작
thr_index = np.arange(1, thresholds.shape[0], 5)
print(f"샘플 추출을 위한 임곗값 배열의 Index : {thr_index}")
print(f"샘플 index로 추출한 임곗값 : {np.round(thresholds[thr_index], 2)}")

# 5 steps 단위로 추출된 임곗값에 따른 FPR, TPR 값
print(f"샘플 임곗값별 FPR : {np.round(fprs[thr_index], 3)}")
print(f"샘플 임곗값별 TPR : {np.round(tprs[thr_index], 3)}")

In [ ]:
def roc_curve_plot(y_test, pred_proba_c1) :
    # 임곗값에 따른 FPR, TPR 값을 반환 받음
    fprs, tprs, thresholds = roc_curve(y_test, pred_proba_c1)

    # ROC Curve를 plot 곡선으로 그림
    plt.plot(fprs, tprs, label="ROC")
    # 가운데 대각선 직선을 그림
    plt.plot([0, 1], [0, 1], "k--", label="Random")

    # FPR X cnrdml Scale을 0.1 단위로 변경, X,Y 축명 설정 등
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))
    plt.xlabel("FPR(1 - Sensitivity)"); plt.ylabel("TPR(Recall)")
    plt.legend()
    plt.show()

roc_curve_plot(y_test, lr_clf.predict_proba(X_test)[:,1])

In [ ]:
from sklearn.metrics import roc_auc_score

# pred = lr_clf.predict(X_test)
# roc_score = roc_auc_score(y_test, pred)

pred_proba = lr_clf.predict_proba(X_test)[:, 1]
roc_score = roc_auc_score(y_test, pred_proba)
print(f"ROC AUC 값 : {roc_score:.4f}")